***Important*** DO NOT CLEAR THE OUTPUT OF THIS NOTEBOOK AFTER EXECUTION!!!

In [1]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [2]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  PRIMARY_WORKER_COUNT  SECONDARY_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-0016  GCE       2                                             RUNNING  us-central1-a


# Imports & Setup

In [3]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Jan 21 11:44 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [5]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [6]:
spark

In [7]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = 'hw3ir322' 
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if "parquet" in b.name:
#         print(b.name)
        paths.append(full_path+b.name)

***GCP setup is complete!*** If you got here without any errors you've earned 10 out of the 35 points of this part.

# Building an inverted index

Here, we read the entire corpus to an rdd, directly from Google Storage Bucket and use your code from Colab to construct an inverted index.

In [8]:
parquetFile = spark.read.parquet(*paths)
doc_anchor_pairs = parquetFile.select("anchor_text", "id").rdd

We will count the number of pages to make sure we are looking at the entire corpus. The number of pages should be more than 6M

In [9]:
# Count number of wiki pages
parquetFile.count()

6348910

Let's import the inverted index module. Note that you need to use the staff-provided version called `inverted_index_gcp.py`, which contains helper functions to writing and reading the posting files similar to the Colab version, but with writing done to a Google Cloud Storage bucket.

In [10]:
# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
!ls inverted_index_gcp.py

inverted_index_gcp.py


In [11]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [12]:
from inverted_index_gcp import InvertedIndex

**YOUR TASK (10 POINTS)**: Use your implementation of `word_count`, `reduce_word_counts`, `calculate_df`, and `partition_postings_and_write` functions from Colab to build an inverted index for all of English Wikipedia in under 2 hours.

A few notes: 
1. The number of corpus stopwords below is a bit bigger than the colab version since we are working on the whole corpus and not just on one file.
2. You need to slightly modify your implementation of  `partition_postings_and_write` because the signature of `InvertedIndex.write_a_posting_list` has changed and now includes an additional argument called `bucket_name` for the target bucket. See the module for more details.
3. You are not allowed to change any of the code not coming from Colab. 

In [13]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ['category', 'references', 'also”', 'links', 'extrenal',
                 'first', 'see', 'new', 'two', 'list', 'may', 'one', 'district',
                 'including', 'became', 'however', 'com', 'many', 'began',
                 'make', 'made', 'part', 'would', 'people', 'second', 'also',
                 'following', 'history', 'thumb', 'external']

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)
stemmer = PorterStemmer()
NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

# PLACE YOUR CODE HERE

def partition_postings_and_write(postings, base_dir):
  bucket_rdd = postings.map(lambda x: (token2bucket_id(x[0]), x)).groupByKey()

  def write_bucket(b_w_pl):
      bucket_id, word_posting_pairs = b_w_pl
      return InvertedIndex.write_a_posting_list((bucket_id, list(word_posting_pairs)),base_dir,bucket_name)

  posting_locs_list = bucket_rdd.map(write_bucket)

  return posting_locs_list


def reduce_word_counts(unsorted_pl):
  ''' Returns a sorted posting list by wiki_id.
  Parameters:
  -----------
    unsorted_pl: list of tuples
      A list of (wiki_id, tf) tuples 
  Returns:
  --------
    list of tuples
      A sorted posting list.
  '''
  # YOUR CODE HERE
  return sorted(unsorted_pl, key = lambda x: x[0])

def calculate_df(postings):
  ''' Takes a posting list RDD and calculate the df for each token.
  Parameters:
  -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each element is a (token, df) pair.
  '''
  # YOUR CODE HERE
  return postings.map(lambda token: (token[0], len(token[1])))

def tokenize(text):
  stemmer = PorterStemmer()
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  stemmed = [stemmer.stem(token) for token in tokens if token not in all_stopwords]
  return stemmed



def stemmed_word_count(text, id):
  tokens = tokenize(text)
  token_counts = Counter(tokens)  # Calculate term frequencies efficiently
  res_list = [
        (token, (id, count))
        for token, count in token_counts.items()
        if token not in all_stopwords
    ]
  return res_list

In [14]:
new_doc_anchor_pairs = doc_anchor_pairs.flatMap(lambda pair: [(p[1], p[0]) for p in pair[0]])

In [15]:
# Apply the parse_anchor function to each element in doc_anchor_pairs and flatten the result
new_pairs = doc_anchor_pairs.flatMap(lambda pair: [(p[1], p[0]) for p in pair[0]])

# Apply the word_count function to each element of tmp_pairs and flatten the result
word_counts_anchor = new_pairs.flatMap(lambda x: stemmed_word_count(x[0], x[1]))

# Group the word counts by key and apply reduce_word_counts to the values
posting_anchor = word_counts_anchor.groupByKey().mapValues(reduce_word_counts)

def new_count(posting):
    counts = Counter(p[0] for p in posting)
    return list(counts.items())

# Apply the group function to the values of each key in posting_anchor
postings = posting_anchor.map(lambda x: (x[0], new_count(x[1])))

In [ ]:
w2df = calculate_df(postings)
w2df_dict = w2df.collectAsMap()

In [19]:
# partition posting lists and write out
_ = partition_postings_and_write(postings, "anchor").collect()

In [20]:
# collect all posting lists locations into one super-set
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='anchor'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs[k].extend(v)

Putting it all together

In [21]:
# Create inverted index instance
inverted = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted.posting_locs = super_posting_locs
# Add the token - df dictionary to the inverted index
inverted.df = w2df_dict
# write the global stats out
inverted.write_index('.', 'index')
# upload to gs
index_src = "index.pkl"
index_dst = f'gs://{bucket_name}/anchor_stemmed/{index_src}'
!gsutil cp $index_src $index_dst

Copying file://index.pkl [Content-Type=application/octet-stream]...
\ [1 files][ 85.8 MiB/ 85.8 MiB]                                                
Operation completed over 1 objects/85.8 MiB.                                     


In [22]:
!gsutil ls -lh $index_dst

 85.85 MiB  2025-01-21T13:33:16Z  gs://hw3ir322/anchor_stemmed/index.pkl
TOTAL: 1 objects, 90020092 bytes (85.85 MiB)


25/01/21 14:23:49 INFO org.apache.spark.SparkEnv: Registering MapOutputTracker
25/01/21 14:23:49 INFO org.apache.spark.SparkEnv: Registering BlockManagerMaster
25/01/21 14:23:49 INFO org.apache.spark.SparkEnv: Registering BlockManagerMasterHeartbeat
25/01/21 14:23:49 INFO org.apache.spark.SparkEnv: Registering OutputCommitCoordinator
25/01/21 14:23:50 ERROR org.apache.spark.deploy.yarn.ApplicationMaster: Uncaught exception: 
org.apache.hadoop.yarn.exceptions.InvalidApplicationMasterRequestException: Application doesn't exist in cache appattempt_1737459814609_0004_000001
	at org.apache.hadoop.yarn.server.resourcemanager.ApplicationMasterService.throwApplicationDoesNotExistInCacheException(ApplicationMasterService.java:361)
	at org.apache.hadoop.yarn.server.resourcemanager.ApplicationMasterService.registerApplicationMaster(ApplicationMasterService.java:259)
	at org.apache.hadoop.yarn.api.impl.pb.service.ApplicationMasterProtocolPBServiceImpl.registerApplicationMaster(ApplicationMasterPro

In [19]:
print(sc)  # Should not be None
print(spark)  # Should not be None


<SparkContext master=yarn appName=PySparkShell>


### currently not working

In [15]:
doc_anchor_pairs.take(10)

[Row(anchor_text=[Row(id=1176764, text='Tactical Air Command'), Row(id=91416, text='Victoria County'), Row(id=136747, text='Victoria, Texas'), Row(id=23814944, text='USGS'), Row(id=185235, text='Air Force Base'), Row(id=32090, text='United States Air Force'), Row(id=29810, text='Texas'), Row(id=91416, text='Victoria County'), Row(id=136747, text='Victoria'), Row(id=32927, text='World War II'), Row(id=1176764, text='Tactical Air Command'), Row(id=325329, text='Cold War'), Row(id=136815, text='Georgetown'), Row(id=23869026, text='U.S. Army Air Corps'), Row(id=1875937, text='Brooks Field'), Row(id=53848, text='San Antonio'), Row(id=763002, text='AT-6'), Row(id=1484086, text='Guadalupe River'), Row(id=18717338, text='$'), Row(id=373962, text='War Department'), Row(id=1029642, text='flight officers'), Row(id=6120180, text='warrant officers'), Row(id=201930, text='second lieutenants'), Row(id=60098, text='Pearl Harbor'), Row(id=1468292, text='WACs'), Row(id=763002, text='AT-6'), Row(id=7211,

In [ ]:
even_id_pair = doc_anchor_pairs.filter(lambda x: x[0] % 2 == 0)
uneven_id_pair = doc_anchor_pairs.filter(lambda x: x[0] % 2 != 0)

In [23]:
anchor_doc_lengths_dict = {}

def len_tokens(id, text):
  og_tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  tokens = [x for x in og_tokens if x not in all_stopwords]
  return (id, len(tokens))

# Apply the function to our RDD

anchor_len_pairs = new_pairs.map(lambda x: len_tokens(x[1], x[0]))

print(type(anchor_len_pairs))

anchor_doc_lengths_dict = anchor_len_pairs.collect()

# saving the dict as pickle file on the cluster
with open('anchor_doc_lengths.pickle', 'wb') as f:
    pickle.dump(anchor_doc_lengths_dict, f)

src = "text_doc_lengths.pickle"
dest = f'gs://{bucket_name}/anchor_stemmed/{src}'

!gsutil cp $src $dest

<class 'pyspark.rdd.PipelinedRDD'>


AttributeError: 'SparkSession' object has no attribute 'driver'